In [34]:
import sys
sys.path.insert(0, '..')

from pandas import read_csv

df = read_csv('../data/schneider.csv')

/tmp/ipykernel_9826/77839716.py:6: DtypeWarning: Columns (5,6,14,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = read_csv('../data/schneider.csv')


VERSION FUNCTIONAL

In [48]:
import torch
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from utils.schneider import *

model_name = "all-MiniLM-L6-v2"

def underscore_ngrams(df):
    preprocessed_df = df.copy()
    
    # Perform preprocessing steps on the 'processed_data' column
    underscore_ngrams_list = list(map(lambda x : x.replace(" ", "_"), ngrams_list))
    ngram_replacements = dict(zip(ngrams_list, underscore_ngrams_list))
    preprocessed_df['processed_data'] = preprocessed_df['processed_data'].replace(ngram_replacements, regex=True)
    
    return preprocessed_df

def restore_ngrams(vocabulary):
    postprocessed_vocab = []
    for keyword in vocabulary:
        space_keyword = keyword.replace("_"," ")
        if space_keyword in ngrams_list:
            keyword = space_keyword
        postprocessed_vocab.append(keyword)
    return postprocessed_vocab

def keybert_vocabulary(df):
    preprocessed_df = df.copy()

    # Preprocess documents by replacing custom n-grams with single tokens containing underscores
    if ngrams_list:
        preprocessed_df = underscore_ngrams(preprocessed_df)

    # Extract the list of documents from the 'processed_data' column
    docs = preprocessed_df["processed_data"].astype(str).tolist()

    # Initialize a GPU-enabled SentenceTransformer model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = SentenceTransformer(model_name, device=device)

    # Extract keywords using KeyBERT
    kw_model = KeyBERT(model=model)
    keywords = kw_model.extract_keywords(docs, **keybert_kwargs)
    
    # Flatten the list of lists and remove duplicates to create the vocabulary
    vocabulary = list(set([word for sublist in keywords for word, _ in sublist]))

    # Postprocess extracted keywords by replacing single tokens with original n-grams
    if ngrams_list:
        postprocessed_vocab = restore_ngrams(vocabulary)
    else:
        postprocessed_vocab = vocabulary

    return postprocessed_vocab

test = keybert_vocabulary(df_preprocessed)
len(test)


10807

Recup du DF Processed

In [53]:
from utils.schneider import countries_to_update, text_data_column, words_to_filter, replacements, ngrams_list, keybert_kwargs, bertopic_kwargs, more_stopwords
from preprocessing.dataLoaders.petition_data_loader import PetitionDataLoader
from preprocessing.preprocessing import Preprocessor
from vocabulary.vocabulary import VocabularyCreator
from clustering.clustering import ClusteringMethod
from sklearn.feature_extraction.text import CountVectorizer
import nltk

nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(more_stopwords)

model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"

df = read_csv("../data/petitions_fr_combi20233.csv")
petitionsDataLoader = PetitionDataLoader(df)
preprocessing = Preprocessor(
    petitionsDataLoader, 
    ['title', 'description'],
    # words_to_filter,
    # replacements
)

df_preprocessed = preprocessing.preprocess(filter_rows= False, replace_words= False)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rachidj/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [54]:
vocabulary_creator = VocabularyCreator(
    model_name,
    # ngrams_list,
    **keybert_kwargs
)
vocabulary_list = vocabulary_creator.keybert_vocabulary(df_preprocessed)
print(len(vocabulary_list))

45284


In [55]:
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance

from clustering.clustering import ClusteringMethod

clustering = ClusteringMethod(model_name= model_name)

bertopic_kwargs={
    # "nr_topics": 40,
    "ctfidf_model": ClassTfidfTransformer(reduce_frequent_words=True),
    "representation_model": [KeyBERTInspired(), MaximalMarginalRelevance(diversity=0.3)], # possible values : KeyBERTInspired() or MaximalMarginalRelevance(diversity=0.5)
    "top_n_words":10,
    "min_topic_size":50,
    "calculate_probabilities": True,
    "n_gram_range": (1,3),
    'vectorizer_model': CountVectorizer(
                    vocabulary=vocabulary_list, 
                    stop_words=stopwords, 
                    lowercase=True, 
                    ngram_range=(1, 3)
                )
}

topics, probs, topic_model, embeddings = clustering.run_bertopic(
    df= df_preprocessed,
    **bertopic_kwargs
    )

clustering.save('../models/model_petitions_fr')

Batches: 100%|██████████| 2592/2592 [06:38<00:00,  6.51it/s]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

/home/rachidj/miniconda3/envs/torchgpu/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['electric', 'tip', 'top'] not in stop_words.
  warnings.warn(
/home/rachidj/miniconda3/envs/torchgpu/lib/python3.8/site-packages/bertopic/vectorizers/_ctfidf.py:69: RuntimeWarning: divide by zero encountered in divide
  idf = np.log((avg_nr_samples / df)+1)


In [56]:
topic_model.visualize_topics()

In [57]:
topic_model.visualize_barchart(top_n_topics=65)

In [58]:
hierarchical_topics = topic_model.hierarchical_topics(df_preprocessed['processed_data'])
topic_model.visualize_hierarchy(hierarchical_topics=hierarchical_topics)

100%|██████████| 154/154 [00:41<00:00,  3.68it/s]
